In [1]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # use normalized 4bit float for QLoRA
    bnb_4bit_compute_dtype=torch.float16 # can use torch.bfloat16 on newer gpus
)

# Model

Download official Llama 2 from Meta, selected 7b but downloaded CodeLlama-7b-Python

Use https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/convert_llama_weights_to_hf.py to convert to hf format

# Dataset
```
from datasets import load_dataset
dataset_save_path = '../dataset/'
data = load_dataset("Abirate/english_quotes", split='train').to_json(f"{dataset_save_path}english_quotes.jsonl")
```

In [2]:
from transformers import LlamaTokenizer, LlamaForCausalLM

model_path = '../model/'

model = LlamaForCausalLM.from_pretrained(
    model_path, quantization_config=bnb_config, device_map='auto',
)

tokenizer = LlamaTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
model.config.use_cache = False # for training
model.config.pretraining_tp = 1

In [4]:
from peft import LoraConfig, get_peft_model

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

config = LoraConfig(
    r=16,
    lora_alpha=32, 
    lora_dropout=0.1, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model) # half of 7b due to 4 bit quantization 

trainable params: 8388608 || all params: 3508801536 || trainable%: 0.23907331075678143


In [5]:
from datasets import load_dataset

dataset_path = '../dataset/english_quotes.jsonl'

data = load_dataset("json", data_files=dataset_path, split='train[:10%]') # use only 10% due to OOM on 16gb gpu
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)
training_data = data.with_format("torch", device='cuda')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/251 [00:00<?, ? examples/s]

In [6]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=training_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4, # effective batch size of 1x4 = 4, hence ~60 steps required for 250 training samples
        num_train_epochs=1,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="./outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
trainer.train()

Step,Training Loss
1,2.047000
2,2.456800
3,1.665600
4,2.087000
5,2.186800
6,1.488700
7,2.384800
8,1.634300
9,1.936600
10,2.953800


TrainOutput(global_step=62, training_loss=1.8574943157934374, metrics={'train_runtime': 70.5855, 'train_samples_per_second': 3.556, 'train_steps_per_second': 0.878, 'total_flos': 365942612238336.0, 'train_loss': 1.8574943157934374, 'epoch': 0.99})

In [7]:
torch.cuda.max_memory_allocated() / 1024 / 1024 / 1024

10.024258136749268